In [2]:
from helium10Scrapper.path import PROJECT_ROOT

In [111]:
import requests
import json
import pandas as pd
import time

# Black Box

## create black box query

In [5]:
f = open(PROJECT_ROOT.joinpath('black_box_filter.01.json'))
all_cat_filter = json.load(f)
f.close()

In [6]:
accountId = 1545862407
bearer_token = "Bearer TdS2Gt40ufi07PZYT6-mnSP0iBs-psd-1H4mgQYEFlePWrDApxSJ3SR1XVFJ55tZb89d2b24a069dd45736f5a3e8846f326"
r = requests.post(
    f"https://research-tools.helium10.com/api/blackbox/v1/search/products?accountId={accountId}",
    json = all_cat_filter,
    headers={"Authorization":f"Bearer {bearer_token}"}
)

In [15]:
black_box_query_creation_response = {}
if r.status_code == 201:
    black_box_query_creation_response = r.json()


black_box_query_id = ""
try:
    black_box_query_id = black_box_query_creation_response["data"]["id"]
except KeyError:
    raise("Error occrued when creating black box query")

In [63]:
black_box_query_creation_response["data"]["id"]

8346471

In [64]:
black_box_query_creation_response["data"]

{'id': 8346471,
 'searchId': '8ea592ea3b7287ebfd5b3bd9da744f5b',
 'marketplaceId': 1,
 'searchType': 1,
 'filters': {'reviewCountTo': 30,
  'monthlyRevenueFrom': 1000000,
  'category': ['194',
   '1242',
   '2029',
   '5246',
   '5972',
   '7008',
   '21314',
   '814579',
   '16027',
   '16145',
   '19959',
   '21988',
   '20629',
   '22781',
   '25102',
   '26926',
   '28512',
   '31986',
   '29841',
   '46677',
   '46680',
   '47637',
   '48411',
   '49397',
   '49966',
   '50131',
   '57553',
   '59808',
   '60643']},
 'sortBy': 'default',
 'createdAt': '2023-10-15T11:59:08.000000Z'}

## get the results of the query

In [16]:
r = requests.get(
    f"https://research-tools.helium10.com/api/blackbox/v1/search/products/{black_box_query_id}/results?",
    params = {
        "accountId":accountId,
        "page":1,
        "per_page":1000
    },
    headers={"Authorization":f"Bearer {bearer_token}"}
)

In [20]:
black_box_result_dict = r.json()

try:
    table_data = black_box_result_dict["data"]["tableData"]
except KeyError:
    raise("Error occrued when requesting for black box query result")

In [71]:
data = []
for data_dict in table_data:
    try:
        data.append(["8346471", data_dict["asin"], data_dict["id"]])
    except KeyError:
        continue

In [74]:
black_box_result_tb = pd.DataFrame(data, columns = ["query_id", "asin", "id"])
black_box_result_tb.to_csv(PROJECT_ROOT.joinpath("black_box_result.01.csv"), index=False)

In [22]:
f = open(PROJECT_ROOT.joinpath("black_box_result.01.json"), "w")
json.dump(table_data, f, indent=4)
f.close()

# cerebro

## create query

In [26]:
def create_cerebro_query(product_asin):
    cerebro_create_query_endpoint = f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single"
    params = {
        "accountId":accountId,
    }
    bearer_token = "Bearer TdS2Gt40ufi07PZYT6-mnSP0iBs-psd-1H4mgQYEFlePWrDApxSJ3SR1XVFJ55tZb89d2b24a069dd45736f5a3e8846f326"
    body = {
        "marketplace": "ATVPDKIKX0DER",
        "productId": product_asin,
        "adminSearch": False,
        "exactProduct": False
    }
    headers = {"Authorization":f"Bearer {bearer_token}"}
    r = requests.post(
        url = cerebro_create_query_endpoint,
        params=params,
        headers=headers,
        json=body
    )
    return r

In [112]:
len(table_data)

200

In [ ]:
cerebro_query_id_dict = {}
for product in table_data[20:]:
    print(asin, end = "..., ")
    asin = product["asin"]
    r = create_cerebro_query(asin)
    print(r.status_code, end = ", sleeping...\n")
    cerebro_query_id_dict[asin] = r
    time.sleep(5)

In [118]:
len(cerebro_query_id_dict.keys())

180

In [119]:
output_tmp_dict = {}
for asin, response in cerebro_query_id_dict.items():
    output_tmp_dict[asin] = response.json()

f = open(PROJECT_ROOT.joinpath("cerebro_query_id_dict.03.json"), "w")
json.dump(output_tmp_dict, f, indent=4)
f.close()

## get cerebro query result

In [41]:
def get_cerebro_query_result(cerebro_query_id):
    cerebro_get_query_endpoint = f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single/{cerebro_query_id}/data"
    params = {
        "accountId":accountId,
        "include-all":0,
        "include-any":1,
        "page":1,
        "per_page":50,
        "sort":"OrganicPosition"
    }

    bearer_token = "Bearer TdS2Gt40ufi07PZYT6-mnSP0iBs-psd-1H4mgQYEFlePWrDApxSJ3SR1XVFJ55tZb89d2b24a069dd45736f5a3e8846f326"
    headers = {"Authorization":f"Bearer {bearer_token}"}
    r = requests.get(cerebro_get_query_endpoint, params=params, headers=headers)
    return r

In [122]:
cerebro_result_dict = {}
for asin, response in cerebro_query_id_dict.items():
    print(asin, end = "\r")
    cerebro_query_id = ""
    try:
        cerebro_query_id = response.json()["data"]["id"]
    except KeyError:
        print("no result for this asin")
        continue

    cerebro_result_dict[asin] = get_cerebro_query_result(cerebro_query_id)
    print(cerebro_result_dict[asin].status_code, end = "\r")
    time.sleep(5)

no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin
no result for this asin


In [123]:
data = []
for asin, response in cerebro_result_dict.items():
    serialized_response = response.json()
    try:
        search_id = serialized_response["data"]["searchId"]
        for organic_rank, record in enumerate(serialized_response["data"]["tableData"][:20]):
                data.append([
                    asin,
                    search_id,
                    organic_rank + 1,
                    record["organicPosition"],
                    record["phraseId"],
                    record["phrase"]
                ])
    except KeyError as e:
        print(str(e))
        continue

In [124]:
cerebro_result_tb = pd.DataFrame(
    data,
    columns=["asin", "search_id", "organic_rank", "organic_position", "phrase_id", "phrase"]
)
cerebro_result_tb.to_csv(PROJECT_ROOT.joinpath("cerebro_result.03.csv"), index=False)

In [125]:
output_tmp_dict = {}
for asin, response in cerebro_query_id_dict.items():
    output_tmp_dict["asin"] = response.json()

f = open(PROJECT_ROOT.joinpath("cerebro_query_result.03.json"), "w")
json.dump(output_tmp_dict, f, indent=4)
f.close()

## get bearer token

In [ ]:
accountId = 1545862407
get_token_url = f"https://members.helium10.com/api/v1/site/token?accountId={accountId}"
bearer_token = "Bearer TdS2Gt40ufi07PZYT6-mnSP0iBs-psd-1H4mgQYEFlePWrDApxSJ3SR1XVFJ55tZb89d2b24a069dd45736f5a3e8846f326"
r = requests.get(get_token_url, headers={"Authorization":f"Bearer {bearer_token}"})

In [ ]:
accountId = 1545862407
get_token_url = f"https://members.helium10.com/api/v1/site/token?accountId={accountId}"
bearer_token = "Bearer TdS2Gt40ufi07PZYT6-mnSP0iBs-psd-1H4mgQYEFlePWrDApxSJ3SR1XVFJ55tZb89d2b24a069dd45736f5a3e8846f326"
r = requests.get(get_token_url, headers={"Authorization":f"Bearer {bearer_token}"})
new_token = r.json()["data"]["token"]